This notebook demonstrates some use cases for gathering more or less structured data from the internet. We will extract top university rankings, profiles of a scientific community and world bank data.

## Part One: University Rankings

In the first part we will extract structured information from HTML tables and merge them into a single dataframe. The tables are located on the webpage [shanghairanking.com](www.shanghairanking.com/) where the top universities are ranked upon several scores.

In [3]:
from IPython.display import HTML
HTML('<iframe src=http://www.shanghairanking.com/ARWU2003.html width=900 height=650></iframe>')

First the necessary packages will be loaded:

In [2]:
import urllib2
from pandas.io import wb
import pandas as pd
import re
from bs4 import BeautifulSoup
import time
import pickle
import pycountry

The next part is using the main url as a starting point and then identifies all links we have to crawl for the tabular data.

In [375]:
seed = 'http://www.shanghairanking.com/'
response = urllib2.urlopen(seed)
html = response.read()
seedsoup = BeautifulSoup(html)
url_list = sorted([list(set(seed + link.get('href') 
           for link in seedsoup.find_all('a', href=re.compile(r'^ARWU[0-9]{4}'))))][0])
url_list

['http://www.shanghairanking.com/ARWU2003.html',
 'http://www.shanghairanking.com/ARWU2004.html',
 'http://www.shanghairanking.com/ARWU2005.html',
 'http://www.shanghairanking.com/ARWU2006.html',
 'http://www.shanghairanking.com/ARWU2007.html',
 'http://www.shanghairanking.com/ARWU2008.html',
 'http://www.shanghairanking.com/ARWU2009.html',
 'http://www.shanghairanking.com/ARWU2010.html',
 'http://www.shanghairanking.com/ARWU2011.html',
 'http://www.shanghairanking.com/ARWU2012.html',
 'http://www.shanghairanking.com/ARWU2013.html',
 'http://www.shanghairanking.com/ARWU2014.html']

Now we write a function to automate the table extraction for every url we gathered above. Extracting the raw tabular data is pretty easy. The hard part is readjusting the dataframe to assure data consistency over all years. Therefore, several steps like renaming and conversion from string to numeric values are included in the function. 

In [376]:
def get_rankings(urls):
    rankings  = pd.DataFrame()
    
    # reading in every url in the list created above
    for url in urls:
        try:
            html = urllib2.urlopen(url).read()
        except:
            time.sleep(30)
            html = urllib2.urlopen(url).read()
        
        # as the country flags are not contained in the tables, we will extract the country
        # from the flag image url for each university
        html_flags = re.sub(r'(<img.*flag/)(.*)(.png.*</a>)', r'<div>\2</div>', html)
        
        df = pd.read_html(html_flags, header=0)[0]
        year = re.findall(r'[0-9]{4}', url)[0]
        
        #several adjustments for data consistency
        if year == '2003':
            df.rename(columns={'Score on  Nobel  HiCi  N&S PUB  Faculty':'Nobel', 
                               'Unnamed: 5': 'HiCi', 'Unnamed: 6': 'N&S', 
                               'Unnamed: 7': 'PUB', "Unnamed: 8": "Faculty"},
                                  inplace=True)
            df['Total Score'] = df['Total Score'].convert_objects(convert_numeric=True)
        elif year == '2004':
            df.rename(columns={'Score on  Alumni  Award  HiCi  N&S PUB  PCP':'Alumni', 
                               'Unnamed: 5': 'Award', 'Unnamed: 6': 'HiCi', 
                               'Unnamed: 7': 'N&S', "Unnamed: 8": "PUB", "Unnamed: 9" : "PCP" },
                              inplace=True)
            df['Total Score'] = df['Total Score'].convert_objects(convert_numeric=True)
        else: 
            df.rename(columns={'Score on  Alumni  Award  HiCi  N&S  PUB  PCP':'Alumni', 
                               'Unnamed: 6': 'Award', 'Unnamed: 7': 'HiCi', 
                               'Unnamed: 8': 'N&S', "Unnamed: 9": "PUB",
                               "Unnamed: 10" : "PCP", "Institution*": "Institution" },
                              inplace=True) 
            df['Total Score'] = df['Total Score'].convert_objects(convert_numeric=True)
            df['National Rank'] = df['National Rank'].convert_objects(convert_numeric=True)            
            
        df.rename(columns={'Alumni':'Alumni' + year , 'Award': 'Award'  + year, 
                           'Nobel': 'Nobel'  + year, 'HiCi': 'HiCi'  + year, 
                           'N&S': 'N&S'  + year,  'Faculty': 'Faculty'  + year,
                           'PUB': 'PUB'  + year,  'PCP': 'PCP'  + year,  
                           "Country  /Region": "Country/Region"  + year,
                            "Total Score" : "Total Score"  + year,
                            "National Rank": "National Rank"  + year,
                            "World Rank": "World Rank"  + year}, inplace=True) 
        df.set_index('Institution', inplace=True)
        
        # this code is merging all data frames using university names as identifiers
        if len(rankings) == 0:
            rankings = df
        else:
            rankings = pd.merge(rankings, df, how='outer', 
                       left_index="True", right_index="True")
    return rankings
       

Now the function can be applied to create the combined dataframe.

In [377]:
df = get_rankings(url_list)

Python can handle almost every data format. In this case we will write the output to an Excel-Spreadsheet.

In [54]:
writer = pd.ExcelWriter('rankings.xlsx')
df[sorted(df.columns)].to_excel(writer,'Sheet1')


In the spreadsheet, rows represent all universities and columns the information for every year. The sheet can be downloaded via the followling link:
[Shanghai-Rankings](https://www.dropbox.com/s/p1wli8cok83vv9j/rankings.xlsx?dl=1).    

Note however that the sheet is not 100% accurate as there are still some inconsistencies within the shanghai-data. This is actually a good example for real life data extraction: it is messy!!!

## Part Two: Leopoldina Member Profiles

In this part we will extract information from members of the scientific community [Leopoldina](http://www.leopoldina.org). This time the data are not structured within a table and have to be collected within single webpages for each member.

In [8]:
from IPython.display import HTML
HTML('<iframe src=http://www.leopoldina.org width=900 height=650></iframe>')

Starting off we again determine the seed page to start crawling url's. There appear to be 1534 member profiles (at the time of writing).


In [640]:
seed = 'http://www.leopoldina.org/de/mitglieder/mitgliederverzeichnis/itemsperpage/100/'

This is an example for a single member profile:

In [10]:
from IPython.display import HTML
HTML('<iframe src=http://www.leopoldina.org/de/mitglieder/mitgliederverzeichnis/member/1141/ \
width=900 height=650></iframe>')

Not all 1534 member profiles are listed on one single url though.
By inspecting the html code of the seed page, we can identify the structure for each subpage containing the member url's. After that, we create a list with every member url. 

In [265]:
leopoldina_urls = []
for i in xrange(1,17):
    url = seed + 'page/' + str(i)
    response = urllib2.urlopen(url)
    html = response.read()
    soup = BeautifulSoup(html)
    for e in soup.find_all(class_='more-link'):
        link = e.find('a', href=re.compile(r'mitgliederverzeichnis/member/.'))
        if str(type(link)) == "<class 'bs4.element.Tag'>":
            leopoldina_urls.append(seed[:26] + link.get('href'))
            
len(scientists)

1532

Two links appear to be broken, so we are left with 1532 web pages to crawl. The following function enables iterating over each member profile. We are interested in several attributes like *country*, *career* and *memberships* as well as PDF-Files containg CV's for every member. 
Again, real life data are messy and finding the attributes for all members requires flexible code. Therefore, different wordings of attributes are considered and possible error messages (e.g. if there is no PDF.File for a specific member) will be catched.

In [453]:
def get_scientists(url_list):
    leopoldina = {}
    pdfs = 0
    with open('logfile.log', 'wb') as f:

        for link in url_list:
            f.write(link.encode('utf-8') + " wird verarbeitet.. \n") 
            try:
                html = urllib2.urlopen(link).read()
            except:
                time.sleep(10)
                html = urllib2.urlopen(link).read()
            soup = BeautifulSoup(html)

            name = soup.find(class_='text').find('h1').text
            leopoldina[name] = {}
            cols = soup.find_all(class_='col2')
            leopoldina[name]['wahljahr'] = cols[0].text
            leopoldina[name]['sektion'] = cols[1].text
            leopoldina[name]['stadt'] = cols[2].text
            leopoldina[name]['land'] = cols[3].text

            if soup.find('a', class_='downloadLinkZeile'):
                cv_url = soup.find('a', class_='downloadLinkZeile').get('href')
                try:
                    pdf_file = urllib2.urlopen(seed[:26] + cv_url).read()
                except:
                    time.sleep(10)
                    pdf_file = urllib2.urlopen(seed[:26] + cv_url).read()
                with open(name + '_cv.pdf' , 'wb') as pdf:
                    pdf.write(pdf_file)
                pdfs += 1
                f.write('PDF gefunden und gespeichert. \n')
            if soup.find('h3', text='Forschung'):
                forschung = soup.find('h3', text='Forschung')
                forschung_string = ""
                forschung_text = forschung.findNextSiblings()
                for text in forschung_text:
                    forschung_string += text.text
                leopoldina[name]['forschung'] = forschung_string
          
            for i in ['Werdegang', 'Career']:  
                if soup.find('h3', text=i):
                    werdegang = soup.find('h3', text=i)
                    werdegang_string = ""
                    werdegang_text = werdegang.findNext()
                    try:
                        for text in werdegang_text:
                            werdegang_string += unicode(text.text)
                            werdegang_string += " \n"
                    except AttributeError:
                        werdegang_string = werdegang_text.text
                    leopoldina[name]['werdegang'] = werdegang_string
                    break
                    
            for i in ['Auszeichnungen und Mitgliedschaften',
                      'Auszeichnungen und Mitgliedschaften (Auswahl)', 
                      'Honours and Memberships']:
                if soup.find('h3', text=i):
                    am = soup.find('h3', text=i)
                    am_string = ""
                    am_text = am.findNext()
                    try:
                        for text in am_text:
                            am_string += unicode(text.text)
                            am_string += " \n"
                    except AttributeError:
                        am_string = am_text.text
                    leopoldina[name]['auszeichnungen/mitgliedschaften'] = am_string
                    break

            f.write(name.encode('utf-8') + " gespeichert. \n\n") 
        f.write("\nInsgesamt " + unicode(len(leopoldina.keys())) + 
                " Profile und " + str(pdfs) + " PDF's gespeichert.")
        with open('leopoldina.p', 'wb') as d:
            pickle.dump(test, d)
        return leopoldina
    

Finally, we can apply the function and crawl every web page to extract member information (this takes some time!).

In [454]:
leopoldina = get_scientists(leopoldina_urls)

In [461]:
len(leopoldina)

1530

At the end, we are left with 1530 member profiles as 2 links again appeared to be broken. The information are stored within a python dictionary but could be extracted with a single line of code. A Log-File has been written to the current working directory, containing the crawling progress. Also, if available, a CV in PDF format is automatically downloaded and named after the corresponding scientist.

This output shows data for the scientist *Dorairajan Balasubramanian*:

In [460]:
for k, v in leopoldina['Dorairajan Balasubramanian'].items():
    print k+":", "\n", v, "\n"

sektion: 
Ophthalmologie, Oto-Rhino-Laryngologie, Stomatologie 

land: 
Indien 

wahljahr: 
2009 

forschung: 
More recently, he has been able to suggest one of the functions of the glaucoma-associated protein optineurin and how mutations in it lead to retinal ganglion cell death. 

stadt: 
Hyderabad 

auszeichnungen/mitgliedschaften: 
Elected Fellow of Indian Academy of Sciences, Bangalore, India 
Elected Fellow of Indian National Science Academy, New Delhi 
Elected Fellow of National Academy of Sciences India, Allahabad 
Elected Fellow, American Association for the Advancement of Science (AAAS), Washington DC, USA 
Elected Fellow, Academy of Sciences of the Developing World (earlier called the Third World Academy of Sciences, TWAS), Trieste, Italy 
Elected Fellow, Mauritius Academy of Science and Technology 
Elected Fellow, National Academy of Sciences Leopoldina, Germany 


werdegang: 
2007-2010 President of the Indian Academy of Sciences, Bangalore, India  
since 1998 Director of R

It is possible to store python objects on disk by using the ````Pickle```` package. An example file for the member profiles can be downloaded here:    
[Leopold-Dictionary](https://www.dropbox.com/s/3icbnrxp10g7fx8/leopoldina.p?dl=1)

The Log-File for the crawling can be downloaded here:   
[Log-File](https://www.dropbox.com/s/4vw0oxiile1tvjc/logfile.log?dl=1)

## Part Three: World Bank API

For the last part we will make use of a direct connection to the World Bank API to extract data. Accessing API's generally is much more convenient then web scraping, as the data are highly structured and you don't have to deal with problems like messy html code.

Using pandas io features, we can search the world bank api for any expressions or (if already known) variable names. Here we are interested in high technology exports and their share in GDP.

In [3]:
wb.search('high-technology').iloc[:,:2][:5]

,id,name
8342,TX.VAL.TECH.CD,High-technology exports (current US$)
8343,TX.VAL.TECH.MF.ZS,High-technology exports (% of manufactured exp...


In [601]:
wb.search('gdp').iloc[:,:2][:5]

,id,name
694,5.51.01.10.gdp,Per capita GDP growth
696,6.0.GDP_current,GDP (current US$)
697,6.0.GDP_growth,GDP growth (annual %)
698,6.0.GDP_usd,GDP (constant 2005 US$)
699,6.0.GDPpc,GDP per capita (2011 US$)


After identifying the target variables, we can easily download the corresponding data directly from the world bank databank and specify the time span.

In [8]:
dat = wb.download(indicator=['TX.VAL.TECH.CD', '6.0.GDP_current']
                  , start=2005, end=2015, country =['all']).dropna()


The result is a Data-Frame:

In [9]:
dat.head()

TX.VAL.TECH.CD  6.0.GDP_current
country   year                                 
Argentina 2012      1945836328     6.031530e+11
          2011      1978069865     5.577273e+11
          2010      1648289776     4.627038e+11
          2009      1546820304     3.784962e+11
          2008      1948517138     4.060037e+11

Computing the high technology share in GDP only requires a simple division of the two columns:

In [12]:
dat['share'] = dat['TX.VAL.TECH.CD'] / dat['6.0.GDP_current']
dat.sort(columns='share', ascending=False).head(10)

TX.VAL.TECH.CD  6.0.GDP_current     share
country    year                                           
Panama     2011      4769250391     3.327050e+10  0.143348
Costa Rica 2007      2530343873     2.632200e+10  0.096130
           2006      2090631242     2.252646e+10  0.092808
           2005      1776832950     1.996489e+10  0.088998
           2008      2426063976     2.983117e+10  0.081326
           2009      1791400685     2.938269e+10  0.060968
           2011      2511568834     4.123730e+10  0.060905
           2010      2193335855     3.629833e+10  0.060425
           2012      2719083321     4.537479e+10  0.059925
Mexico     2005     32400251888     8.663465e+11  0.037399

As you can see, accessing world bank information with Python is pretty straight forward. However, not every API is as easy to handle and well documented.

Finally, we might want to limit an API request to only contain data for specific countries. Of course, we could just pass a list of countries. But what if there are many countries? And what if the list of countries has to be adapted for the API request?

Let's say we are interested only in countries which appear in the shanghai rankings from *Part One*. The strings containing the image-url's are not structured at all and again a perfect example of messy real life data. First, we extract all strings from the DataFrame we created before:

In [ ]:
countries = sorted([list(set(val for col in df.filter(like="Country")
                                 for val in df[col].dropna()))][0])

Regarding a proper format, the World Bank API for example accepts  ISO-Country-Codes. So we have to clean up the list of countries by using several regular expressions:

In [ ]:
countries_clean = []
for c in countries:
    c  = re.sub(r'(China-)(.*)', r'\2', c)
    c  = re.sub(r'(.*),(.*)', r'\1', c)
    c  = re.sub(r'([A-Z].*[a-z])([A-Z].*)', r'\1 \2', c)
    c = c.replace('Taiwan', 'Taiwan, Province of China')
    c = c.replace('Iran', 'Iran, Islamic Republic of')
    c = c.replace('Russia', 'Russian Federation')
    c = c.replace('Czech', 'Czech Republic')
    c = c.replace('South Korea', 'Korea, Republic of')
    if len(c) > 3:   
        countries_clean.append(c)
countries_clean = sorted(list(set(countries_clean)))

After that, we convert the country strings into ISO codes:

In [635]:
countrie_codes = {}
for country in pycountry.countries:
    countrie_codes[country.name] = country.alpha3
codes = [countrie_codes.get(country, 'Unknown code') for country in countries_clean]
print len(codes)

45


Overall we are left with a list of 45 codes. The first 10 ISO codes and the corresponding country names are printed below:

In [604]:
for country, code in zip(countries_clean, codes)[-10:]:
    print country, code

Singapore SGP
Slovenia SVN
South Africa ZAF
Spain ESP
Sweden SWE
Switzerland CHE
Taiwan, Province of China TWN
Turkey TUR
United Kingdom GBR
United States USA


Finally we can make use of the list for another API request. This time we are interested in charges for intellectual property:

In [555]:
wb.search('property').iloc[:,:2][:5]

,id,name
1513,BM.GSR.ROYL.CD,"Charges for the use of intellectual property, ..."
1594,BX.GSR.ROYL.CD,"Charges for the use of intellectual property, ..."
5336,IC.PRP.COST.PROP.ZS,Cost of registering property (% of property va...
5337,IC.PRP.DURS,Time required to register property (days)
5338,IC.PRP.PROC,Procedures to register property (number)


For the download we hand over the list with country codes. For unknown reasons, Taiwan had to be excluded because the API raised errors for this country.

In [623]:
df_prop = wb.download(indicator='BM.GSR.ROYL.CD', start ='2013', end='2013',
                country = [code for code in codes if code != "TWN"]).dropna()

And the result is another DataFrame:

In [624]:
df_prop.sort(ascending=False).head()

,,BM.GSR.ROYL.CD
country,year,
United States,2013,39016000000
United Kingdom,2013,9036715541
Turkey,2013,785000000
Sweden,2013,2246965121
Spain,2013,2096370149
